# Improved LLM Plans
Original feature, Only one mental Keggle Mental health information used.
Then, based on information, how should I imporve my model,
In inital Training, gpt model4, and we have serveral information inside.
Next approch, I like to Hugging Datasets and Mental health datasets

Currently, feature we just populate randomly.
Next what, I want is text related llm topic clustering,
Based on token, clustering.


In [1]:
# Hugging Datasets called
raw_data_path = "raw_data"
cleaned_data_path= "cleaned_data"



In [2]:
import pandas as pd
# Inital Raw Data Used.

def clean_keggle_df(data_path):
    df = pd.read_csv(data_path)
    df = df[["questionText", "topics", "re_diagnosis","clean_answer_text"]]
    # Lower case
    df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    # remove non-world
    df = df.replace(to_replace=r'[^\w\s]', value="", regex=True)
    # remove number
    df = df.replace(to_replace=r'\d', value='', regex=True)

    return df

def clean_hugging_df(data_path):
    df = pd.read_csv(data_path)
    df = df[["questionTitle", "questionText", "topic", "answerText"]]
    df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    df["questionText"] = df["questionTitle"].fillna('') + " " + df["questionText"].fillna('')
    df = df.replace(to_replace=r'[^\w\s]', value="", regex=True)
    df = df.replace(to_replace=r'\d', value='', regex=True)
    df = df[["questionText", "topic", "answerText"]]
    return df

keggle_df = clean_keggle_df(f"{raw_data_path}/counsel_cleaned.csv")   
print(keggle_df.shape)
keggle_df.to_csv(f"{cleaned_data_path}/cleaned_counsel.csv")
hugging_df = clean_hugging_df(f"{raw_data_path}/huggin_counsel_chat.csv")
print(hugging_df.shape)
hugging_df

(1373, 4)
(2775, 3)


C:\Users\ykim\AppData\Local\Temp\ipykernel_28620\3358003086.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
C:\Users\ykim\AppData\Local\Temp\ipykernel_28620\3358003086.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


,questionText,topic,answerText
0,do i have too many issues for counseling i hav...,depression,it is very common for people to have multiple ...
1,do i have too many issues for counseling i hav...,depression,ive never heard of someone having too many iss...
2,do i have too many issues for counseling i hav...,depression,absolutely not i strongly recommending workin...
3,do i have too many issues for counseling i hav...,depression,let me start by saying there are never too man...
4,do i have too many issues for counseling i hav...,depression,i just want to acknowledge you for the courage...
...,...,...,...
2770,are some clients more difficult than others wh...,counselingfundamentals,although many clients have the capacity to be ...
2771,are some clients more difficult than others wh...,counselingfundamentals,i usually dont label a client as difficult bec...
2772,are some clients more difficult than others wh...,counselingfundamentals,dang right heh heh and correct me if im wrong...
2773,are some clients more difficult than others wh...,counselingfundamentals,yes just like some relationships outside of ou...


In [ ]:
import pandas as pd

# Read File Information
hugging_df = pd.read_csv(f"{cleaned_data_path}/cleaned_hugging.csv")
counsel_df = pd.read_csv(f"{cleaned_data_path}/cleaned_counsel.csv")

# Change Column Name
hugging_df.rename(columns={"questionText": "question_text",
                           "topic": "topics", 
                           "answerText": "answer_text"}, inplace=True)
hugging_df.drop(columns=['Unnamed: 0'], inplace=True)

print("After renaming:", hugging_df.columns)
# Drop unused column
counsel_df.drop(columns=['Unnamed: 0', 're_diagnosis'], inplace=True)
print(counsel_df.columns)
counsel_df.rename(columns={"questionText": "question_text", 
                           "clean_answer_text": "answer_text"}, inplace=True)

# Select target column
hugging_df = hugging_df[["question_text", "topics", "answer_text"]]
counsel_df = counsel_df[["question_text", "topics", "answer_text"]]

# Concat Column
combined_dataset = pd.concat([hugging_df, counsel_df], ignore_index=True)

# Combined Output
combined_dataset.to_csv(f"{cleaned_data_path}/combined_output.csv")


After renaming: Index(['questionTitle', 'question_text', 'topics', 'answer_text'], dtype='object')
Index(['questionText', 'topics', 'clean_answer_text'], dtype='object')
Index(['question_text', 'topics', 'answer_text'], dtype='object')
Index(['question_text', 'topics', 'answer_text'], dtype='object')
Index(['question_text', 'topics', 'answer_text'], dtype='object')


In [3]:
# Chat Promt, Design.
combined_dataset = pd.read_csv(f"{cleaned_data_path}/combined_output.csv")
print(combined_dataset.columns)
print(combined_dataset.head)

Index(['Unnamed: 0', 'question_text', 'topics', 'answer_text'], dtype='object')
<bound method NDFrame.head of       Unnamed: 0                                      question_text  \
0              0  i have so many issues to address i have a hist...   
1              1  i have so many issues to address i have a hist...   
2              2  i have so many issues to address i have a hist...   
3              3  i have so many issues to address i have a hist...   
4              4  i have so many issues to address i have a hist...   
...          ...                                                ...   
4143        4143  my grandsons stepmother sends him to school wi...   
4144        4144  my boyfriend is in recovery from drug addictio...   
4145        4145  the birth mother attempted suicide several tim...   
4146        4146  i think adult life is making him depressed and...   
4147        4147  i just took a job that requires me to travel f...   

                                     

In [4]:
# NLTK test
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
text = "This is an example. Here is another sentence."
sentences = sent_tokenize(text)
print(sentences)


['This is an example.', 'Here is another sentence.']


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ykim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [7]:
# Cleaned Combined Datasets too shorts and too long
import re
from nltk.tokenize import word_tokenize

def is_noisy(text: str) -> bool:
    if re.search(r'[가-힣A-Za-z]', text) is None:
        return True
    cleaned = re.sub(r'[^\w\s]', '', text) 
    if len(cleaned) == 0 or len(cleaned) < len(text) * 0.02:  
        return True
    return False

def clean_combined_dataset(df):
    def token_count(text):
        tokens = sent_tokenize(text)
        return len(tokens)
    # Apply the token_count function to calculate the number of tokens in questions and answers
    df = df.dropna()
    df['q_token_count'] = df['question_text'].apply(token_count)
    df['a_token_count'] = df['answer_text'].apply(token_count)
    
    return df
# print(combined_dataset.columns)
print(f"Before: {combined_dataset.shape}")
cleaned_combined_df = clean_combined_dataset(combined_dataset)
cleaned_combined_df.to_csv(f"{cleaned_data_path}/cleaned_combined.csv")
print(f"After: {cleaned_combined_df.shape}")

Before: (4148, 4)
After: (3985, 6)


C:\Users\ykim\AppData\Local\Temp\ipykernel_28620\1488087169.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['q_token_count'] = df['question_text'].apply(token_count)
C:\Users\ykim\AppData\Local\Temp\ipykernel_28620\1488087169.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['a_token_count'] = df['answer_text'].apply(token_count)


In [8]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("cleaned_data/cleaned_combined.csv", encoding="utf-8")
df = df.drop(columns=[col for col in df.columns if 'Unnamed' in col])

df['a_text_length'] = df['answer_text'].astype(str).apply(len)

print(df['a_text_length'].describe())
MAX_LENGTH_COUNT =  1200
MIN_LENGTH_COUNT =  200

filter_df = df[(df['a_text_length'] <= MAX_LENGTH_COUNT) &
               (df['a_text_length'] >= MIN_LENGTH_COUNT)] .copy()
filter_df.to_csv("cleaned_data/filtered.csv")


count    3985.000000
mean      975.754580
std       672.451943
min         9.000000
25%       508.000000
50%       802.000000
75%      1236.000000
max      5380.000000
Name: a_text_length, dtype: float64


In [9]:
import torch
import torch.nn.functional as F

# Filter Drop parts
df = pd.read_csv("cleaned_data/filtered.csv", encoding="utf-8")
df = df.drop(columns=[col for col in df.columns if 'Unnamed' in col])

df_unique_questions = df.drop_duplicates(subset='question_text')
df_unique_questions.to_csv("cleaned_data/03_unique_data.csv")

# Re_Sample_information
df_sampled = df_unique_questions.sample(n=300, random_state = 42)

In [ ]:
from transformers import pipeline
import pandas as pd
from tqdm import tqdm

# Zero-shot classification pipeline 
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Improved T/F Classfication
def classify_tf_zero_shot(text):
    labels = [
        "This response contains logical reasoning.",
        "This response provides emotional support."
    ]
    try:
        result = classifier(
            text,
            candidate_labels=labels,
            hypothesis_template="{}"
        )
        top_label = result['labels'][0]
        score = result['scores'][0]
        # T/F mapping
        if "logical reasoning" in top_label:
            tf_type = "T"
        elif "emotional support" in top_label:
            tf_type = "F"
        else:
            tf_type = "Unknown"
        return pd.Series([tf_type, score])
    except Exception as e:
        return pd.Series([None, None])  # 오류 발생 시

# Handle Data frame
tqdm.pandas()
df_sampled[['tf_type_zero_shot', 'tf_score']] = df_sampled['answer_text'].progress_apply(classify_tf_zero_shot)

c:\Users\ykim\Desktop\Personal\Mental-Health-AI-Driven-System-Project\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
100%|██████████| 300/300 [04:17<00:00,  1.17it/s]


In [10]:
import pandas as pd
# Point for
# df_sampled.to_csv("cleaned_data/accurate_sample.csv")
df_sampled = pd.read_csv("cleaned_data/accurate_sample.csv")

df_sampled_F = df_sampled[df_sampled['tf_type_zero_shot'] == "F"]
df_sampled_F.to_csv("cleaned_data/accurate_F_sample.csv")
print(df_sampled_F.shape)
df_sampled_T = df_sampled[df_sampled['tf_type_zero_shot'] == "T"]
df_sampled_T.to_csv("cleaned_data/accurate_T_sample.csv")
print(df_sampled_T.shape)


(202, 9)
(98, 9)


In [ ]:
from sklearn.utils import resample
import pandas as pd

# Create a working copy
df_labeled = df_sampled.copy()

# Select 'F' samples with low confidence (likely mislabeled)
df_pseudo_T = df_labeled[
    (df_labeled['tf_type_zero_shot'] == "F") &
    (df_labeled['tf_score'] < 0.55) &
    (df_labeled['tf_score'] > 0.30)
].copy()

# Force-label these as 'T'
df_pseudo_T['tf_type_zero_shot'] = "T"

# Combine original T + pseudo-labeled T
df_minority_extended = pd.concat([
    df_labeled[df_labeled['tf_type_zero_shot'] == "T"],
    df_pseudo_T
])

# Resample T up to match F count
df_majority = df_labeled[df_labeled['tf_type_zero_shot'] == "F"]
df_minority_upsampled = resample(
    df_minority_extended,
    replace=True,
    n_samples=len(df_majority),
    random_state=42
)

# Combine into a balanced dataset
df_balanced = pd.concat([df_majority, df_minority_upsampled])

In [13]:
from sklearn.model_selection import train_test_split

# Train/Test Split

df_labeled = df_balanced[['answer_text', 'tf_type_zero_shot']]

# Split into training and validation sets (stratified)
X_train, X_val, y_train, y_val = train_test_split(
    df_labeled['answer_text'],
    df_labeled['tf_type_zero_shot'],
    test_size=0.2,
    random_state=42,
    stratify=df_labeled['tf_type_zero_shot']
)

# Feature Extraction (TF-IDF)
from sklearn.feature_extraction.text import TfidfVectorizer

# Use bi-grams and limit to 5000 features
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))

# Fit on training, transform both train and validation
X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)


# Classifier Training (Logistic Regression)

from sklearn.linear_model import LogisticRegression

# Train logistic regression model
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_vec, y_train)


from sklearn.metrics import classification_report

# Predict on validation set
y_pred = clf.predict(X_val_vec)
print(classification_report(y_val, y_pred))

# Predict on full dataset with probabilities
proba = clf.predict_proba(vectorizer.transform(df['answer_text']))
df['predicted_tf_type'] = clf.predict(vectorizer.transform(df['answer_text']))
df['proba_F'] = proba[:, 0]
df['proba_T'] = proba[:, 1]

df.to_csv("cleaned_data/prob_all.csv")


              precision    recall  f1-score   support

           F       0.79      0.80      0.80        41
           T       0.79      0.78      0.78        40

    accuracy                           0.79        81
   macro avg       0.79      0.79      0.79        81
weighted avg       0.79      0.79      0.79        81



In [15]:
# Load Predictions and Deduplicate
df = pd.read_csv("cleaned_data/prob_all.csv")
df = df.drop_duplicates(subset=['answer_text', 'question_text'])
print("After deduplication:", df.shape)

# Filtered by class
df_T_confident = df[df['predicted_tf_type'] == 'T'].sort_values(by='proba_T', ascending=False).head(300)
print("df_T_confident", df_T_confident.shape)
df_F_confident = df[df['predicted_tf_type'] == 'F'].sort_values(by='proba_F', ascending=False).head(300)
print("df_F_confident", df_F_confident.shape)


df_balanced_concat = pd.concat([df_T_confident, df_F_confident]).reset_index(drop=True)


After deduplication: (2149, 10)
df_T_confident (300, 10)
df_F_confident (300, 10)


In [ ]:
import json
df_balanced_concat.to_csv("cleaned_data/04_final_600.csv")
df_balanced_concat = pd.read_csv("cleaned_data/04_final_600.csv")
print(df_balanced_concat.count)

def get_system_prompt(tf_type):
    if tf_type == "T":
        return "You are a mental health assistant. Respond either with logical reasoning or emotional support depending on the user’s needs."
    else:
        return "You are a mental health assistant. Use emotional support and validation in your response."

jsonl_data = []

for _, row in df_balanced_concat.iterrows():
    system_msg = get_system_prompt(row['predicted_tf_type'])
    user_msg = row['question_text'].strip()
    assistant_msg = row['answer_text'].strip()

    entry = {
        "messages": [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_msg},
            {"role": "assistant", "content": assistant_msg}
        ]
    }
    jsonl_data.append(entry)

# Save

with open("600_Chat_Turning.jsonl", "w", encoding="utf-8") as f:
    for entry in jsonl_data:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")



<bound method DataFrame.count of      Unnamed: 0.3  Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0  \
0               0             0             0        2535   
1               1             1             1         668   
2               2             2             2        1153   
3               3             3             3        1638   
4               4             4             4         371   
..            ...           ...           ...         ...   
595           595           595           595        1260   
596           596           596           596         260   
597           597           597           597         529   
598           598           598           598         132   
599           599           599           599        1918   

                                         question_text  \
0    she refuses to talk to me and told my mom her ...   
1    she refuses to talk to me and told my mom her ...   
2    i have no idea what happened i go places and d...   
3 